In [1]:
%pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 5.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark import SparkConf, pandas as ps
from pyspark.sql import SparkSession, functions as sf, Window
from pyspark.sql.functions import col, collect_list
from pyspark.sql.types import *
import psycopg2
import time
import os
import glob

/usr/local/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
conf = SparkConf().set("spark.driver.memory", "8g")

spark_session = SparkSession\
    .builder\
    .master("local")\
    .config(conf=conf)\
    .appName("ETL") \
    .getOrCreate()

In [4]:
oltp_params = {
    'user': 'admin',
    'password': 'password',
    "driver": "org.postgresql_postgresql-42.7.3.jar"
}

oltp_url = "jdbc:postgresql://rfm-segmentation-oltp-db-1:5432/trans_oltp"

In [5]:
# print(spark_session.sparkContext._jvm.java.sql.DriverManager.getDrivers())
print(spark_session.sparkContext._jsc.sc().listJars())

Vector(spark://cd322f95bf8a:38097/jars/org.checkerframework_checker-qual-3.42.0.jar, spark://cd322f95bf8a:38097/jars/org.postgresql_postgresql-42.7.3.jar)


In [6]:
df = spark_session.read.format('jdbc').option('url', oltp_url).option('dbtable', "trans").option('user', 'admin').option('password', 'password').option('driver', "org.postgresql.Driver").load()

df.show()


+----------+----------+--------------------+--------------+-------------+-----------------+----------------+--------------------+-------+
|  trans_id|trans_date|          total_cost|payment_method|         city|       store_type|discount_applied|           promotion|cust_id|
+----------+----------+--------------------+--------------+-------------+-----------------+----------------+--------------------+-------+
|1000299020|2021-07-03|78.20000000000000...|Mobile Payment|        Miami| Department Store|           false|Discount on Selec...|  33877|
|1000342674|2021-10-25|11.60000000000000...|Mobile Payment|San Francisco|      Supermarket|            true|BOGO (Buy One Get...|  33877|
|1000806142|2020-05-25|90.06000000000000...|    Debit Card|      Chicago|   Warehouse Club|            true|                None| 629479|
|1000008294|2020-07-27|90.77000000000000...|          Cash|San Francisco| Department Store|            true|BOGO (Buy One Get...| 143811|
|1000641795|2020-05-22|68.82000000

In [7]:
product_dim = spark_session.read.format('jdbc').option('url', oltp_url).option('dbtable', "product").option('user', 'admin').option('password', 'password').option('driver', "org.postgresql.Driver").load()
customer_dim = spark_session.read.format('jdbc').option('url', oltp_url).option('dbtable', "cust").option('user', 'admin').option('password', 'password').option('driver', "org.postgresql.Driver").load()
tran_prod_fact = spark_session.read.format('jdbc').option('url', oltp_url).option('dbtable', "trans_prod").option('user', 'admin').option('password', 'password').option('driver', "org.postgresql.Driver").load()
tran_dim = spark_session.read.format('jdbc').option('url', oltp_url).option('dbtable', "trans").option('user', 'admin').option('password', 'password').option('driver', "org.postgresql.Driver").load()


In [8]:
tran_prod_fact.describe()


DataFrame[summary: string, product: string, trans_id: string, product_id: string]

In [25]:
customer_dim.show()


+--------------------+-----------------+-------+
|       customer_name|customer_category|cust_id|
+--------------------+-----------------+-------+
|     Michelle Acosta|          Student|      1|
|       Carolyn Brown|   Senior Citizen|      2|
|        Victor Weeks|         Teenager|      3|
|        Lori Pearson|         Teenager|      4|
|           Amy Davis|          Student|      5|
|      Joshua Coleman|      Young Adult|      6|
|      Nathan Russell|   Senior Citizen|      7|
|        Wayne Fields|      Young Adult|      8|
|      Robert Jackson|      Young Adult|      9|
|Samantha Sherman DVM|      Young Adult|     10|
|         Katie Jones|          Student|     11|
|Alexandria Wilkerson|     Professional|     12|
|      Christina Cook|      Middle-Aged|     13|
|          Kent Jones|          Student|     14|
|      Melissa Morgan|          Retiree|     15|
|     Jessica Mullins|      Young Adult|     16|
|   Mr. Jerome Fisher|          Student|     17|
|  Rachael Mcculloug

In [19]:
product_dim.product_id = product_dim.rename
product_dim.show()


+--------------+----------+
|       product|product_id|
+--------------+----------+
|    Lawn Mower|         1|
|       Pickles|         2|
|        Shrimp|         3|
|          Eggs|         4|
|       Ketchup|         5|
|          Tuna|         6|
|         Water|         7|
|     Olive Oil|         8|
|         Broom|         9|
|      Potatoes|        10|
|        Orange|        11|
|   Light Bulbs|        12|
| Peanut Butter|        13|
|    Trash Bags|        14|
|        Butter|        15|
|        Cereal|        16|
|       Mustard|        17|
| Ironing Board|        18|
|Cleaning Spray|        19|
|           Tea|        20|
+--------------+----------+
only showing top 20 rows



In [11]:
tran_dim.describe()

DataFrame[summary: string, trans_id: string, total_cost: string, payment_method: string, city: string, store_type: string, promotion: string, cust_id: string]

In [32]:
tran_prod_fact2 = tran_prod_fact.join(other=tran_dim, on='trans_id', how='left').select(['trans_id', 'product_id', 'trans_date', 'cust_id'])
tran_prod_fact2.show()

+----------+----------+----------+-------+
|  trans_id|product_id|trans_date|cust_id|
+----------+----------+----------+-------+
|1000001491|         2|2022-10-29| 315863|
|1000010824|         2|2022-01-06| 423272|
|1000025357|         2|2023-11-30| 189014|
|1000027766|         1|2020-11-11| 398088|
|1000028663|         1|2023-08-22| 513663|
|1000032837|         1|2023-03-24| 333029|
|1000036123|         1|2020-11-26|  27619|
|1000038805|         1|2021-09-23| 378306|
|1000040863|         2|2022-04-28| 434641|
|1000043591|         1|2021-07-26|  65245|
|1000050636|         2|2023-08-20|  38189|
|1000050860|         2|2020-08-30|  30304|
|1000055410|         1|2021-07-31|  89984|
|1000055491|         1|2024-03-09| 431868|
|1000059460|         1|2021-09-05| 318968|
|1000060350|         1|2021-05-22| 228746|
|1000062227|         2|2022-12-03|  98458|
|1000074421|         1|2020-09-18|  54856|
|1000091287|         2|2023-02-11| 107278|
|1000099710|         2|2020-06-10| 245685|
+----------

In [18]:
tran_dim2= tran_dim.drop('cust_id').drop('trans_date')
tran_dim2.show(15)

+----------+--------------------+--------------+-------------+-----------------+----------------+--------------------+
|  trans_id|          total_cost|payment_method|         city|       store_type|discount_applied|           promotion|
+----------+--------------------+--------------+-------------+-----------------+----------------+--------------------+
|1000299020|78.20000000000000...|Mobile Payment|        Miami| Department Store|           false|Discount on Selec...|
|1000342674|11.60000000000000...|Mobile Payment|San Francisco|      Supermarket|            true|BOGO (Buy One Get...|
|1000806142|90.06000000000000...|    Debit Card|      Chicago|   Warehouse Club|            true|                None|
|1000008294|90.77000000000000...|          Cash|San Francisco| Department Store|            true|BOGO (Buy One Get...|
|1000641795|68.82000000000000...|Mobile Payment|       Boston|  Specialty Store|            true|Discount on Selec...|
|1000933527|94.91000000000000...|          Cash|

In [22]:
olap_params = {
    'host': 'rfm-segmentation-olap-db-1',
    'database': 'trans_olap',
    'user': 'admin',
    'password': 'password',
    'port': '5432'
}



In [23]:
def insert_from_df(df, table, conn=psycopg2.connect(**olap_params)):
    tmp = './tmp'
    df.write.csv(tmp,mode='overwrite')
    tmp_files = glob.glob('./tmp/*.csv')
    print(tmp_files)
    cursor = conn.cursor()

    try:
        for file in tmp_files:
            with open(file, 'r') as f:
                cursor.copy_from(f, table, sep=",", )
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as e:
        [os.remove(file) for file in tmp_files]
        print("Error: %s" % e)
        conn.rollback()
        cursor.close()
        return 1
    print("Table %s copied from dataframe" % table)
    cursor.execute(F"Select * FROM {table} LIMIT 5;")
    cursor.fetchall()
    cursor.close()
    [os.remove(file) for file in tmp_files]


In [35]:
# insert_from_df(df=tran_dim2, table="trans_dim")
# insert_from_df(df=customer_dim.select(["cust_id","customer_name","customer_category"]), table="cust_dim")
# insert_from_df(df=product_dim.select(["product_id", "product"]), table="prod_dim")
insert_from_df(df=tran_prod_fact2.select(['trans_id','product_id','trans_date','cust_id']) , table="trans_prod_fact")


['./tmp/part-00000-c617dad0-b923-4c7b-9885-44654a8f1b3f-c000.csv']
Table trans_prod_fact copied from dataframe
